<a href="https://colab.research.google.com/github/ahathaway821/COVID-19-Fact-Checking/blob/master/data/politifact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4
!pip install soupsieve
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import Request, urlopen
from datetime import date

In [34]:
num_pages = 15
data = []
for i in range(1, num_pages + 1):
    url = 'https://www.politifact.com/factchecks/list/?page=' + str(i) + '&category=coronavirus'
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    soup = BeautifulSoup(webpage, 'html.parser')
    facts = soup.find_all('li', attrs={'class':'o-listicle__item'})

    for fact in facts:
        who_posted = fact.find('a', attrs={'class':'m-statement__name'}).text.strip('\n')
        when_posted = fact.find('div', attrs={'class':'m-statement__desc'}).text.strip('\n')
        date = when_posted.split(" ")[2:5]
        what_posted = fact.find('div', attrs={'class':'m-statement__quote'}).text.strip('\n')
        rating_div = fact.find('div', attrs={'class':'m-statement__meter'})
        rating = rating_div.find('img', attrs={'class':'c-image__thumb'})['alt'].strip('\n')
        data.append({
            'claim': what_posted,
            'posted': " ".join(date),
            'source': who_posted,
            'rating': rating
        })

df = pd.DataFrame(data)
df

,claim,posted,source,rating
0,“She’s just casually carrying a body with 1 ha...,"June 3, 2020",Viral image,false
1,“You see suicide rates are skyrocketing now…”,"May 26, 2020",Tim Murtaugh,false
2,"“According to the CDC, so far this year, Flori...","May 27, 2020",Facebook posts,false
3,Says Nancy Pelosi tried “once again” to includ...,"May 13, 2020",Bloggers,barely-true
4,“The new Dane County lockdown policy… (has) ef...,"May 19, 2020",Scott Manley,barely-true
...,...,...,...,...
436,"Says a ""Clorox bottle claimed it could kill (t...","January 28, 2020",Facebook posts,false
437,"Says the Chinese Communist Party ""will finally...","January 25, 2020",Bloggers,false
438,"""CORONAVIRUS: Reports of 10,000 DEAD in Wuhan,...","January 24, 2020",Facebook posts,pants-fire
439,"Says China ""stole Coronavirus from Canada and ...","January 26, 2020",Bloggers,false


In [0]:
df.to_csv('politifact_data.csv')